<a href="https://colab.research.google.com/github/JiaweiYu1/fasttext/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import re
import os
import codecs

import tensorflow as tf

import time
import datetime

from sklearn.utils import shuffle
import csv
import html

from tensorflow.contrib import slim

import argparse
from datetime import datetime
import pickle
import pprint
import easydict
import random


#tf.enable_eager_execution()

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Colab_Notebooks/fasttext")
cwd = os.getcwd()
print(cwd)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/Colab_Notebooks/fasttext


In [0]:
class fastTextModel():
    """
    A simple implementation of fasttext for text classification
    """
    def __init__(self, sequence_length, num_classes, vocab_size,
                 embedding_size, learning_rate, l2_reg_lambda, epoch, is_training=True,
                 initializer=tf.random_normal_initializer(stddev=0.1)):
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_classes = num_classes
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate
        #self.decay_steps = decay_steps
        #self.decay_rate = decay_rate
        self.epoch = epoch
        self.is_training = is_training
        self.l2_reg_lambda = l2_reg_lambda
        self.initializer = initializer
 
        self.input_x = tf.placeholder(tf.int32, [None, self.sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.int32, [None, self.num_classes], name='input_y')
 
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.instantiate_weight()
        self.logits = self.inference()
        self.loss_val = self.loss()
        self.train_op = self.train()
 
        self.predictions = tf.argmax(self.logits, axis=1, name='predictions')
        correct_prediction = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'), name='accuracy')
 
    def instantiate_weight(self):
        with tf.variable_scope('weights', reuse=tf.AUTO_REUSE):
            self.Embedding = tf.get_variable('Embedding', shape=[self.vocab_size, self.embedding_size],
                                             initializer=self.initializer)
            self.W_projection = tf.get_variable('W_projection', shape=[self.embedding_size, self.num_classes],
                                                initializer=self.initializer)
            self.b_projection = tf.get_variable('b_projection', shape=[self.num_classes])
 
 
    def inference(self):
        with tf.name_scope('embedding'):
            words_embedding = tf.nn.embedding_lookup(self.Embedding, self.input_x)
            self.average_embedding = tf.reduce_mean(words_embedding, axis=1)
 
        logits = tf.matmul(self.average_embedding, self.W_projection) +self.b_projection
 
        return logits
 
 
    def loss(self):
        # loss
        with tf.name_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.logits)
            data_loss = tf.reduce_mean(losses)
            #l2_loss = tf.add_n([tf.nn.l2_loss(cand_var) for cand_var in tf.trainable_variables()
            #                    if 'bias' not in cand_var.name]) * self.l2_reg_lambda
            #data_loss += l2_loss * self.l2_reg_lambda
            return data_loss
 
    def train(self):
        with tf.name_scope('train'):
            #learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step,
            #                                           self.decay_steps, self.decay_rate,
            #                                           staircase=True)
 
            train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step,
                                                      learning_rate=learning_rate, optimizer='Adam')
 
        return train_op

In [4]:

class_size=4
learning_rate=0.1
batch_size=32
#decay_steps=1000
#decay_rate=0.9
sequence_length=467
vocab_size = 1439345
embedding_dim = 10
is_training=True
dropout_keep_prob=1
epoch = 30
batch_size = 4096
l2_reg_lambda = 0.01


fasttext = fastTextModel(sequence_length,
                      class_size,
                      vocab_size,
                      embedding_dim,
                      learning_rate,
                      l2_reg_lambda,
                      epoch,
                      is_training=True,
                    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
def batch_iter(x, y, batch_size=1, shuffle=True):
  data_x = np.array(x)
  data_y = np.array(y)
  data_len = len(x)
  num_batch = int((data_len-1)/batch_size)+1
    
  if shuffle:
    shuffle_indices = np.random.permutation(np.arange(data_len))
    x_shuffle = data_x[shuffle_indices]
    y_shuffle= data_y[shuffle_indices]
    
  else:
    x_shuffle=x
    y_shuffle = y
  for i in range(num_batch):
    start_index = i*batch_size
    end_index = min((i+1)*batch_size, data_len)
    yield (x_shuffle[start_index:end_index], y_shuffle[start_index:end_index])
    
    
    
def fit(train_x, train_y, x_dev, y_dev, batch_size, epoch):
  if not os.path.exists('saves/fasttext'): os.makedirs('saves/fasttext')
  if not os.path.exists('train_logs/fasttext'): os.makedirs('train_logs/fasttext')

  train_steps = 0
  best_val_acc = 0
    
  tf.summary.scalar('val_loss', fasttext.loss_val)
  tf.summary.scalar('val_accuracy', fasttext.accuracy)
  merged = tf.summary.merge_all()
    
  sess = tf.Session()
    
  writer = tf.summary.FileWriter('train_logs/fasttext', sess.graph)
  saver = tf.train.Saver(max_to_keep=10)
  sess.run(tf.global_variables_initializer())
    
  for i in range(epoch):
    start_time = time.time()
    batch_train = batch_iter(train_x, train_y, batch_size)
    for batch_x, batch_y in batch_train:
      train_steps +=1
      feed_dict = {fasttext.input_x:batch_x, fasttext.input_y:batch_y}
      _, train_loss, train_acc = sess.run([fasttext.train_op, fasttext.loss_val, fasttext.accuracy], feed_dict = feed_dict)
        
      if train_steps % 10 ==0:
        feed_dict = {fasttext.input_x:x_dev, fasttext.input_y:y_dev}
        val_loss, val_acc = sess.run([fasttext.loss_val, fasttext.accuracy],feed_dict=feed_dict)
        
        summary = sess.run(merged, feed_dict=feed_dict)
        writer.add_summary(summary, global_step=train_steps)
        
        if val_acc >= best_val_acc:
          best_val_acc = val_acc
          saver.save(sess, "saves/fasttext/model.ckpt", global_step=train_steps)
        msg = 'epoch:%d/%d, train_steps:%d, train_loss:%.4f, trina_acc:%.4f, val_loss:%.4f, val_acc:%.4f'
        print(msg%(i, epoch, train_steps, train_loss, train_acc, val_loss, val_acc))
    
    print("--- %s seconds per epoch ---" % (time.time() - start_time))
        

      
def predict(x):
  sess = tf.Session()
    
  sess.run(tf.global_variables_initializer())
  saver = tf.train.Saver(tf.global_variables())
  ckpt = tf.train.get_checkpoint_state('saves/fasttext/')
  saver.restore(sess, ckpt.model_checkpoint_path)
  #saver.restore(sess, 'saves/fasttext/checkpoint')
    
  feed_dict={fasttext.input_x:x}
  logits = sess.run(fasttext.logits, feed_dict=feed_dict)
  y_pred = np.argmax(logits, 1)
    
  return y_pred

In [0]:
file_to_read = open('data/train_x.pkl', 'rb')

train_x = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/y_train.pkl', 'rb')

y_train = pickle.load(file_to_read)

file_to_read.close()

In [0]:
dev_sample_index = -1 * int(0.1 * float(len(y_train)))
x_train, x_dev = train_x[:dev_sample_index], train_x[dev_sample_index:]
y_train_1, y_dev_1 = y_train[:dev_sample_index], y_train[dev_sample_index:]

In [0]:
y_dev_2 = np.array(y_dev_1).astype(np.int32).tolist()
y_train_2 = np.array(y_train_1).astype(np.int32).tolist()

In [10]:
fit(x_train, y_train_2, x_dev, y_dev_2,batch_size, epoch)

epoch:0/30, train_steps:10, train_loss:1.2861, trina_acc:0.7051, val_loss:1.2719, val_acc:0.5834
epoch:0/30, train_steps:20, train_loss:0.8310, trina_acc:0.8733, val_loss:0.8109, val_acc:0.8509
--- 10.950680017471313 seconds per epoch ---
epoch:1/30, train_steps:30, train_loss:0.3627, trina_acc:0.9155, val_loss:0.4462, val_acc:0.8712
epoch:1/30, train_steps:40, train_loss:0.2455, trina_acc:0.9272, val_loss:0.3211, val_acc:0.8977
epoch:1/30, train_steps:50, train_loss:0.2158, trina_acc:0.9343, val_loss:0.2811, val_acc:0.9070
--- 11.84830117225647 seconds per epoch ---
epoch:2/30, train_steps:60, train_loss:0.1086, trina_acc:0.9688, val_loss:0.2667, val_acc:0.9114
epoch:2/30, train_steps:70, train_loss:0.0933, trina_acc:0.9724, val_loss:0.2585, val_acc:0.9124
epoch:2/30, train_steps:80, train_loss:0.0801, trina_acc:0.9780, val_loss:0.2483, val_acc:0.9172
--- 11.604283571243286 seconds per epoch ---
epoch:3/30, train_steps:90, train_loss:0.0409, trina_acc:0.9922, val_loss:0.2513, val_acc:

In [0]:
file_to_read = open('data/test_x.pkl', 'rb')

test_x = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/test_y.pkl', 'rb')

test_y = pickle.load(file_to_read)

file_to_read.close()

In [13]:
y_predict = predict(test_x)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from saves/fasttext/model.ckpt-150


In [0]:
c_prediction = tf.equal(y_predict, test_y)

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, y_predict)

array([[1754,   46,   62,   38],
       [  18, 1860,   14,    8],
       [  50,   12, 1698,  140],
       [  52,   21,  119, 1708]])

In [16]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, y_predict)
print('The test accuracy is', accuracy)

The test accuracy is 0.9236842105263158
